# Definitive Nerual Network Model

### Import libraries

In [1]:
import numpy as np
import os
import time
from keras.models import model_from_json
from scipy.stats import moment
from keras import backend as K
from keras.models import Sequential
from keras.layers import Flatten, Dense, Input, Activation
from sklearn.model_selection import train_test_split
from random import seed
seed(0)

#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices()[1])

#from keras import backend as K
#K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


## Functions

### Import data

In [2]:
#Format data:
#argumentos del comando
#volumen
#nb boxes
#nb spaces
#volumenes de cajas
#L de cajas
#W de cajas
#H de cajas
#volumenes de espacios
#L de espacios
#W de espacios
#H de espacios
#volumen alcanzado (BSG 500s)

def read_txt(path):
    txt = open(path)
    data = (txt.read()).split('\n')
    txt.close()
    return data

def sample_list(data):
    sample_list = []
    for line in data:
        if ':' in line:
            sample = []
            sample_list.append(sample)
        if line != '':
            sample.append(line)
    return sample_list

def get_features(txt_sample_list):
    args = []
    state_volume = []
    nb_boxes = []
    nb_spaces = []
    boxes_volume = []
    l_boxes = []
    w_boxes = []
    h_boxes = []
    spaces_volume = []
    l_spaces = []
    w_spaces = []
    h_spaces = []
    volume_reached = []

    for file in txt_sample_list:
        for feature in file:
            if len(feature) == 13:
                args.append(feature[0])
                state_volume.append(float(feature[1]))
                nb_boxes.append(float(feature[2]))
                nb_spaces.append(float(feature[3]))
                boxes_volume.append(get_array_features(feature, 4))
                l_boxes.append(get_array_features(feature, 5))
                w_boxes.append(get_array_features(feature, 6))
                h_boxes.append(get_array_features(feature, 7))
                spaces_volume.append(get_array_features(feature, 8))
                l_spaces.append(get_array_features(feature, 9))
                w_spaces.append(get_array_features(feature, 10))
                h_spaces.append(get_array_features(feature, 11))
                volume_reached.append(float(feature[12]))
            else:
                print('Sample', feature[0].split(':')[0], 'has not been imported')
    
    return np.array(args), np.array(state_volume), np.array(nb_boxes), np.array(nb_spaces), np.array(boxes_volume), np.array(l_boxes), np.array(w_boxes), np.array(h_boxes), np.array(spaces_volume), np.array(l_spaces), np.array(w_spaces), np.array(h_spaces), np.array(volume_reached)

def get_array_features(feature, index):
    aux = np.array(feature[index].strip().split(' ')).astype('float32')
    aux = np.array([np.mean(aux), moment(aux, moment=2), moment(aux, moment=3), moment(aux, moment=4), moment(aux, moment=5)])
    return (aux - np.amin(aux)) / (np.amax(aux) - np.amin(aux)) if np.amin(aux) > 0 and np.amax(aux) > 0 else aux

def process_data(state_volume, nb_boxes, nb_spaces, boxes_volume, l_boxes, w_boxes, h_boxes, spaces_volume, l_spaces, w_spaces, h_spaces):
    data = []
    for file in range(0, len(boxes_volume)):
        data.append(np.concatenate((state_volume[file], nb_boxes[file], nb_spaces[file], boxes_volume[file], l_boxes[file], w_boxes[file], h_boxes[file], spaces_volume[file], l_spaces[file], w_spaces[file], h_spaces[file]), axis=None))
    return np.array(data)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

### Start Neural Network with parameters

In [3]:
with K.tf.device('/gpu:0'):
    DATADIR = "trainingset_allinfo/"
    txt_sample_list = []
    for name_txt in os.listdir(DATADIR):
        if ("trainingset_allinfo" in name_txt and ".txt" in name_txt) and '~$' not in name_txt:
            data = read_txt(DATADIR + name_txt)
            txt_sample_list.append(sample_list(data))

    args, state_volume, nb_boxes, nb_spaces, boxes_volume, l_boxes, w_boxes, h_boxes, spaces_volume, l_spaces, w_spaces, h_spaces, volume_reached = get_features(txt_sample_list)
    print('Imported data')
    processed_data = process_data(state_volume, nb_boxes, nb_spaces, boxes_volume, l_boxes, w_boxes, h_boxes, spaces_volume, l_spaces, w_spaces, h_spaces)
    print('Processed data')
    
    epochs = 50
    activation = 'relu'
    random_state = 0
    
    x_train, x_test, y_train, y_test = train_test_split(processed_data, volume_reached, test_size=0.25, random_state=random_state)

    #print(x_train.shape)
    #print(x_test.shape)
    #print(y_train.shape)
    #print(y_test.shape)

    print('Starting Dense Neural Network')
    print('Start time:', time.asctime(time.localtime(time.time())))

    model = Sequential()
    model.add(Dense(2, input_dim=processed_data.shape[1]))
    model.add(Activation(activation))
    model.add(Dense(43))
    model.add(Activation(activation))
    model.add(Dense(1))
    model.add(Activation('linear'))
    model.compile(loss='mae',
                  optimizer='adam',
                  metrics=['mse', root_mean_squared_error,'mae'])

    model.summary()

    scores = model.fit(x_train, y_train, epochs=epochs)
    test_metrics = model.evaluate(x_test, y_test)

    #export_test_metrics(first_it, last_it, test_metrics, ['loss_msre', 'accuracy', 'mse', 'root_mean_squared_error','mae'])
    #export_scores(first_it, last_it, scores)
    print('End time:', time.asctime(time.localtime(time.time())), end='\n\n')
    print('loss:', test_metrics[0])
    print('mse:', test_metrics[1])
    print('rmse:', test_metrics[2])
    print('mae:', test_metrics[3])
    
    # serialize model to JSON
    model_json = model.to_json()
    with open("ann_model_" + activation + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("ann_model_" + activation + ".h5")
    print("Saved model to disk in", os.getcwd())

Sample 15 has not been imported
Sample 24 has not been imported
Sample 29 has not been imported
Sample 32 has not been imported
Sample 45 has not been imported
Sample 50 has not been imported
Sample 52 has not been imported
Sample 61 has not been imported
Sample 71 has not been imported
Sample 90 has not been imported
Sample 95 has not been imported
Sample 99 has not been imported
Sample 110 has not been imported
Sample 147 has not been imported
Sample 155 has not been imported
Sample 157 has not been imported
Sample 158 has not been imported
Sample 164 has not been imported
Sample 177 has not been imported
Sample 178 has not been imported
Sample 183 has not been imported
Sample 196 has not been imported
Sample 202 has not been imported
Sample 208 has not been imported
Sample 217 has not been imported
Sample 225 has not been imported
Sample 226 has not been imported
Sample 234 has not been imported
Sample 243 has not been imported
Sample 247 has not been imported
Sample 261 has not bee

Sample 89 has not been imported
Sample 96 has not been imported
Sample 98 has not been imported
Sample 107 has not been imported
Sample 111 has not been imported
Sample 115 has not been imported
Sample 120 has not been imported
Sample 126 has not been imported
Sample 133 has not been imported
Sample 137 has not been imported
Sample 138 has not been imported
Sample 152 has not been imported
Sample 155 has not been imported
Sample 163 has not been imported
Sample 175 has not been imported
Sample 176 has not been imported
Sample 193 has not been imported
Sample 198 has not been imported
Sample 206 has not been imported
Sample 219 has not been imported
Sample 220 has not been imported
Sample 225 has not been imported
Sample 240 has not been imported
Sample 252 has not been imported
Sample 253 has not been imported
Sample 257 has not been imported
Sample 261 has not been imported
Sample 267 has not been imported
Sample 269 has not been imported
Sample 299 has not been imported
Sample 307 ha

Epoch 1/50
2808/2808 [==============================] - 21s 8ms/step - loss: 0.6728 - mean_squared_error: 0.5568 - root_mean_squared_error: 0.6728 - mean_absolute_error: 0.6728
Epoch 2/50
2808/2808 [==============================] - 1s 356us/step - loss: 0.2317 - mean_squared_error: 0.0964 - root_mean_squared_error: 0.2317 - mean_absolute_error: 0.2317
Epoch 3/50
2808/2808 [==============================] - 1s 194us/step - loss: 0.0552 - mean_squared_error: 0.0062 - root_mean_squared_error: 0.0552 - mean_absolute_error: 0.0552
Epoch 4/50
2808/2808 [==============================] - 1s 194us/step - loss: 0.0275 - mean_squared_error: 0.0020 - root_mean_squared_error: 0.0275 - mean_absolute_error: 0.0275
Epoch 5/50
2808/2808 [==============================] - 1s 244us/step - loss: 0.0232 - mean_squared_error: 0.0014 - root_mean_squared_error: 0.0232 - mean_absolute_error: 0.0232
Epoch 6/50
2808/2808 [==============================] - 1s 237us/step - loss: 0.0185 - mean_squared_error: 8.94

2808/2808 [==============================] - 1s 354us/step - loss: 0.0090 - mean_squared_error: 2.6173e-04 - root_mean_squared_error: 0.0090 - mean_absolute_error: 0.0090
Epoch 46/50
2808/2808 [==============================] - 1s 356us/step - loss: 0.0098 - mean_squared_error: 2.9171e-04 - root_mean_squared_error: 0.0098 - mean_absolute_error: 0.0098
Epoch 47/50
2808/2808 [==============================] - 1s 346us/step - loss: 0.0097 - mean_squared_error: 2.8765e-04 - root_mean_squared_error: 0.0097 - mean_absolute_error: 0.0097
Epoch 48/50
2808/2808 [==============================] - 1s 354us/step - loss: 0.0098 - mean_squared_error: 2.8763e-04 - root_mean_squared_error: 0.0098 - mean_absolute_error: 0.0098
Epoch 49/50
2808/2808 [==============================] - 1s 351us/step - loss: 0.0106 - mean_squared_error: 3.1942e-04 - root_mean_squared_error: 0.0106 - mean_absolute_error: 0.0106
Epoch 50/50
937/937 [==============================] - 0s 265us/step
End time: Mon Oct  7 16:49:5